In [1]:
import datetime
import pandas
import psycopg2
import os
import matplotlib.pyplot as plt

conn = psycopg2.connect(host = '127.0.0.1' , user = 'postgres' , dbname = 'postgres' , password = 'mon3636a')
cur = conn.cursor()
pandas.set_option('display.max_rows',5000)

In [2]:
sma = pandas.read_sql("select date , no , close , sma10 , sma20 from ods.analyze_avg where date_no = (select max(date_no) from work_date) and (close < sma10 or close < sma20) " , con = conn)
FI = pandas.read_sql("select date , no , sum10 , sum20 from ods.analyze_fi_obs where date_no = (select max(date_no) from work_date) and (sum10 > 300000 or sum20 > 300000)" , con = conn)
name = pandas.read_sql("select date , no , name from stock_daily where date = (select max(date) from work_date)\
                 union \
                 select date , no , name from otc_daily where date = (select max(date) from work_date)" , con = conn)

In [8]:
main = pandas.merge(sma,FI , on =['date','no'])
main = main[main['close'] > 0]

main['v10'] = main.apply(lambda x : round((x['sma10'] - x['close']) / x['close'],4)*100  if (x['close'] < x['sma10']) and x['sum10'] > 0 else '' , axis = 1)
main['v20'] = main.apply(lambda x : round((x['sma20'] - x['close']) / x['close'],4)*100 if (x['close'] < x['sma20']) and x['sum20'] > 0 else '' , axis = 1)
main['v10'] = main.apply(lambda x : str(x['v10'])[:4] + '%' if x['v10'] != '' else '' , axis = 1 )
main['v20'] = main.apply(lambda x : str(x['v20'])[:4] + '%' if x['v20'] != '' else '', axis = 1 )
mask_1 = main['v10'] != ''
mask_2 = main['v20'] != ''
main = main[ mask_1 | mask_2 ]
main = main.sort_values(['sum20','sum10'] , ascending = False)

In [9]:
n_main = pandas.merge(main,name , on = ['date','no']).iloc[:,[0,1,9,2,3,4,5,6,7,8]]

In [10]:
query_D = n_main['date'][0]
n_main.to_csv('GOOD_PRICE/GOOD_PRICE_{:0>4}{:0>2}{:0>2}.csv'.format(query_D.year , query_D.month , query_D.day),index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'GOOD_PRICE/GOOD_PRICE_20210720.csv'